In [ ]:
"""
Write a Python function that implements the decision tree learning algorithm for classification. The function should use recursive binary splitting based on entropy and information gain to build a decision tree. It should take a list of examples (each example is a dict of attribute-value pairs) and a list of attribute names as input, and return a nested dictionary representing the decision tree.

Example:
Input:
examples = [
                    {'Outlook': 'Sunny', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak', 'PlayTennis': 'No'},
                    {'Outlook': 'Sunny', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Strong', 'PlayTennis': 'No'},
                    {'Outlook': 'Overcast', 'Temperature': 'Hot', 'Humidity': 'High', 'Wind': 'Weak', 'PlayTennis': 'Yes'},
                    {'Outlook': 'Rain', 'Temperature': 'Mild', 'Humidity': 'High', 'Wind': 'Weak', 'PlayTennis': 'Yes'}
                ],
                attributes = ['Outlook', 'Temperature', 'Humidity', 'Wind']
Output:
{
            'Outlook': {
                'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
                'Overcast': 'Yes',
                'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}
            }
        }
Reasoning:
Using the given examples, the decision tree algorithm determines that 'Outlook' is the best attribute to split the data initially. When 'Outlook' is 'Overcast', the outcome is always 'Yes', so it becomes a leaf node. In cases of 'Sunny' and 'Rain', it further splits based on 'Humidity' and 'Wind', respectively. The resulting tree structure is able to classify the training examples with the attributes 'Outlook', 'Temperature', 'Humidity', and 'Wind'.
"""

In [7]:
import math
from collections import Counter

def entropy(examples: list[dict], target_attr: str) -> float:
    counts = Counter(ex[target_attr] for ex in examples)
    total_examples = len(examples)

    if total_examples == 0:
        return 0

    ent = 0.0
    for count in counts.values():
        probability = count / total_examples
        if probability > 0:
            ent -= probability * math.log2(probability)
    
    return ent

def get_subset(examples: list[dict], attribute: str, value) -> list[dict]:
    return [ex for ex in examples if ex[attribute] == value]

def information_gain(examples: list[dict], attribute: str, target_attr: str) -> float:
    total_entropy = entropy(examples, target_attr)
    values = set(ex[attribute] for ex in examples)
    total_examples = len(examples)
    
    if total_examples == 0:
        return 0

    weighted_entropy = 0.0
    for value in values:
        subset = get_subset(examples, attribute, value)
        subset_size = len(subset)
        
        if subset_size > 0:
            weighted_entropy += (subset_size / total_examples) * entropy(subset, target_attr)
            
    return total_entropy - weighted_entropy

def choose_best_attribute(examples: list[dict], attributes: list[str], target_attr: str) -> str:

    best_gain = -1.0
    best_attr = None
    
    for attr in attributes:
        gain = information_gain(examples, attr, target_attr)
        if gain > best_gain:
            best_gain = gain
            best_attr = attr
            
    return best_attr

def majority_value(examples: list[dict], target_attr: str):

    counts = Counter(ex[target_attr] for ex in examples)
    return counts.most_common(1)[0][0]

def learn_decision_tree(examples: list[dict], attributes: list[str], target_attr: str, parent_majority_class=None):
    
    if not examples:
        return parent_majority_class

    all_target_vals = [ex[target_attr] for ex in examples]
    current_majority = Counter(all_target_vals).most_common(1)[0][0]

    if len(set(all_target_vals)) == 1:
        return all_target_vals[0]

    if not attributes:
        return current_majority

    best_attr = choose_best_attribute(examples, attributes, target_attr)
    tree = {best_attr: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attr]

    for value in set(ex[best_attr] for ex in examples):
        subset = get_subset(examples, best_attr, value)
        subtree = learn_decision_tree(subset, remaining_attributes, target_attr, current_majority)
        tree[best_attr][value] = subtree

    return tree

In [11]:
print(learn_decision_tree([ {'Outlook': 'Sunny', 'Wind': 'Weak', 'PlayTennis': 'No'}, {'Outlook': 'Overcast', 'Wind': 'Strong', 'PlayTennis': 'Yes'}, {'Outlook': 'Rain', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Sunny', 'Wind': 'Strong', 'PlayTennis': 'No'}, {'Outlook': 'Sunny', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Overcast', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Rain', 'Wind': 'Strong', 'PlayTennis': 'No'}, {'Outlook': 'Rain', 'Wind': 'Weak', 'PlayTennis': 'Yes'} ], ['Outlook', 'Wind'], 'PlayTennis'))

{'Outlook': {'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Overcast': 'Yes', 'Sunny': {'Wind': {'Strong': 'No', 'Weak': 'No'}}}}
